Hypothesis: comovement between the stocks 
taking meta, our hypothesis is that given the highest X stocks with highest pearson value, if more than 50% of those stocks is increasing or decreasing along with meta, then the trend will be more likely to continue.
for 2025 1/1, is meta increasing or decreasing?
check out of all the stocks with high pearson, how many are increasing and how many are decreasing
thresold?

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels as sm
from pathlib import Path
import yfinance as yf
from tqdm import tqdm

from correlation_helper import *

In [2]:

num_top = 50
data_path = Path(f'../data/top_{num_top}_companies_by_sector.csv').resolve()
data_path2 = Path(f'../data/beta_matrix_2024-01-01_2025-01-01_top{num_top}.csv')
market_ticker = '^GSPC'
analysis_start = "2024-01-01"
analysis_end = "2025-03-01"


In [3]:
# Import data
data_for_company = pd.read_csv(data_path)#.sample(10, replace=False)
data_for_pearson_matrix = pd.read_csv(data_path2).set_index('Ticker')
target_ticker = 'ISRG'
meta_correlation = data_for_pearson_matrix[[target_ticker]].apply(lambda x: np.abs(x)).sort_values(by=target_ticker,ascending=False)
meta_correlation

,ISRG
Ticker,
ISRG,1.000000
META,0.959858
CHX,0.871288
POWL,0.870351
VRSK,0.857059
...,...
BTSGU,NaN
SIMA,NaN
NYMTI,NaN


In [4]:
num_top = 50
top = meta_correlation[meta_correlation[target_ticker] < 1].dropna()[:num_top]
top

,ISRG
Ticker,
META,0.959858
CHX,0.871288
POWL,0.870351
VRSK,0.857059
NFLX,0.847495
AMZN,0.844116
FAST,0.843164
QCOM,0.842704
IDXX,0.829380


In [5]:
ticker_list = list(top.index)
ticker_list.append(target_ticker)
ticker_list

['META',
 'CHX',
 'POWL',
 'VRSK',
 'NFLX',
 'AMZN',
 'FAST',
 'QCOM',
 'IDXX',
 'LIN',
 'REGCO',
 'CWST',
 'EBAY',
 'EQIX',
 'MPWR',
 'NHPAP',
 'CSWI',
 'MDGL',
 'INFN',
 'SOHOB',
 'AMWD',
 'UFPI',
 'PAYX',
 'JBHT',
 'RVMD',
 'LFUS',
 'ROAD',
 'ROST',
 'VIASP',
 'EXPE',
 'EXC',
 'MKSI',
 'FRPT',
 'TXN',
 'KLAC',
 'GWRS',
 'BWEN',
 'BYRN',
 'COST',
 'CME',
 'TER',
 'CTAS',
 'BCPC',
 'KOSS',
 'ITRI',
 'JJSF',
 'FIP',
 'CACC',
 'ABNB',
 'SSNC',
 'ISRG']

In [6]:
top_correlation_data = yf.download(ticker_list, start=analysis_start, end=analysis_end).dropna(axis=1, how='all')['Close']
top_correlation_data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  51 of 51 completed


Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,134.479996,92.070000,149.929993,145.439133,2.70,5.730000,540.109985,28.478275,199.910934,646.158203,...,42.439999,136.330231,28.230000,18.385307,59.398594,104.230431,163.363083,121.826530,15.615700,236.566528
2024-01-03,133.419998,89.150002,148.470001,141.987198,2.61,6.080000,522.210022,28.379732,192.439255,640.239380,...,40.900002,133.448776,27.990000,18.080336,58.336853,101.225983,160.930908,117.438972,14.131816,235.294113
2024-01-04,133.720001,88.919998,144.570007,142.643753,2.68,6.280000,521.150024,27.416119,193.709976,643.874023,...,40.430000,133.943878,28.030001,17.426830,58.366344,101.056854,158.739990,116.788231,15.161806,236.536713
2024-01-05,135.979996,88.739998,145.240005,139.440521,2.52,5.800000,534.270020,27.564369,190.318115,651.481201,...,40.500000,133.181427,28.379999,17.426830,58.808735,102.240723,159.348038,116.522011,15.371297,232.311935
2024-01-08,140.080002,89.830002,149.100006,141.698715,2.65,6.040000,539.390015,27.346937,191.197159,657.122009,...,41.080002,134.478577,29.330000,17.426830,59.929462,105.344658,162.668198,118.848907,15.441126,231.327850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,144.820007,70.440002,212.710007,165.570007,1.55,26.750000,490.209991,29.540001,247.750000,1035.599976,...,70.190002,135.929993,41.230000,16.507246,88.401123,113.580002,200.740005,106.301147,23.219999,297.010010
2025-02-25,141.550003,71.739998,212.800003,173.589996,1.53,26.370001,479.510010,29.120001,250.820007,1055.660034,...,69.599998,136.449997,41.340000,16.691740,88.201675,111.349998,199.979996,109.480705,23.100000,299.720001
2025-02-26,144.029999,71.150002,214.350006,172.690002,1.57,27.639999,480.519989,28.879999,247.990005,1031.969971,...,73.010002,138.779999,40.500000,16.672319,88.091988,114.809998,198.240005,108.414207,23.350000,288.029999


In [ ]:
def get_interval_end_rows(df, start_date, end_date, interval='M'):
    """
    Extract the last entry for each specified interval within the date range.

    Parameters:
        df (pd.DataFrame): DataFrame indexed by datetime.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        interval (str): Resampling interval ('M' for month-end, 'W' for week-end, 'Q' for quarter-end, etc.)

    Returns:
        pd.DataFrame: Extracted interval-end rows.

    Example:
        >>> df = pd.DataFrame({'price': [10, 20, 30, 40, 50, 60]},
        ...      index=['2024-01-01', '2024-01-15', '2024-01-31',
        ...             '2024-02-10', '2024-02-29', '2024-03-05'])
        >>> get_interval_end_rows(df, '2024-01-01', '2024-02-29', 'M')
                    price
        2024-01-31     30
        2024-02-29     50
    """
    df.index = pd.to_datetime(df.index)
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    df_filtered = df.loc[start_date:end_date]

    interval_end_rows = df_filtered.resample(interval).last().dropna()

    return interval_end_rows

In [56]:
start_month = '2024-11-1'
end_month = '2025-02-28'
interval = '3D'
stock_price_each_month = get_interval_end_rows(top_correlation_data, start_month, end_month, interval)
stock_price_each_month

Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-01,136.460007,91.260002,197.929993,170.567688,1.85,14.680000,417.329987,27.852739,217.140060,876.341736,...,78.750000,139.730103,55.090000,17.262732,69.975426,106.258789,203.415756,122.379517,20.797134,275.103424
2024-11-04,140.910004,99.809998,207.089996,182.455490,1.83,15.940000,458.010010,31.320629,214.770645,898.257507,...,90.930000,138.263565,59.419998,16.970303,73.722687,108.763336,213.391983,135.673416,20.598375,278.748444
2024-11-07,134.610001,100.739998,208.179993,178.386780,1.88,17.500000,450.000000,30.682858,219.130753,942.758301,...,94.459999,143.062241,60.439999,16.904272,74.329018,110.649231,218.672928,133.724579,20.700178,284.590454
2024-11-10,133.199997,100.489998,208.910004,177.401932,1.69,18.379999,439.609985,31.440210,219.761932,931.350952,...,94.510002,139.979523,59.330002,17.036335,73.851906,108.094795,211.218063,132.581116,20.651699,287.526428
2024-11-13,132.500000,96.500000,202.610001,176.844849,1.55,18.320000,445.320007,30.533377,218.761749,906.068848,...,88.519997,140.358627,55.130001,16.762774,72.957344,102.357277,199.643646,128.474625,21.495220,280.416138
2024-11-16,132.190002,95.870003,201.699997,176.158432,1.58,19.059999,445.959991,30.762579,220.975769,918.495117,...,90.099998,140.967194,54.240002,16.908045,73.066673,103.923874,205.659164,128.474625,21.660046,279.976746
2024-11-19,133.259995,94.739998,198.380005,179.371613,1.55,20.309999,460.019989,31.171152,220.004700,954.595276,...,96.849998,142.623291,56.779999,16.951437,74.706726,105.290894,196.745087,129.777161,21.663923,287.166901
2024-11-22,137.339996,97.239998,197.119995,181.490540,1.72,21.309999,473.100006,31.131290,222.412949,962.946045,...,99.980003,145.745911,56.750000,16.663725,75.660934,107.975052,196.546555,132.312668,21.776392,290.332581
2024-11-25,138.279999,89.190002,205.740005,178.754852,1.75,19.049999,494.239990,30.593170,230.463089,960.488708,...,101.139999,154.525177,58.580002,16.536377,76.893456,107.975044,197.727829,134.490189,21.543697,294.157349


In [57]:
stock_growth_between_month = stock_price_each_month.diff(axis=0)
stock_growth_between_month

Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-04,4.449997,8.549995,9.160004,11.887802,-0.02,1.259999,40.680023,3.467890,-2.369415,21.915771,...,12.180000,-1.466537,4.329998,-0.292429,3.747261,2.504547,9.976227,13.293900,-0.198759,3.645020
2024-11-07,-6.300003,0.930000,1.089996,-4.068710,0.05,1.560000,-8.010010,-0.637772,4.360107,44.500793,...,3.529999,4.798676,1.020000,-0.066031,0.606331,1.885895,5.280945,-1.948837,0.101803,5.842010
2024-11-10,-1.410004,-0.250000,0.730011,-0.984848,-0.19,0.879999,-10.390015,0.757353,0.631180,-11.407349,...,0.050003,-3.082718,-1.109997,0.132063,-0.477112,-2.554436,-7.454865,-1.143463,-0.048479,2.935974
2024-11-13,-0.699997,-3.989998,-6.300003,-0.557083,-0.14,-0.059999,5.710022,-0.906834,-1.000183,-25.282104,...,-5.990005,0.379105,-4.200001,-0.273561,-0.894562,-5.737518,-11.574417,-4.106491,0.843521,-7.110291
2024-11-16,-0.309998,-0.629997,-0.910004,-0.686417,0.03,0.740000,0.639984,0.229202,2.214020,12.426270,...,1.580002,0.608566,-0.889999,0.145271,0.109329,1.566597,6.015518,0.000000,0.164825,-0.439392
2024-11-19,1.069992,-1.130005,-3.319992,3.213181,-0.03,1.250000,14.059998,0.408573,-0.971069,36.100159,...,6.750000,1.656097,2.539997,0.043392,1.640053,1.367020,-8.914078,1.302536,0.003878,7.190155
2024-11-22,4.080002,2.500000,-1.260010,2.118927,0.17,1.000000,13.080017,-0.039862,2.408249,8.350769,...,3.130005,3.122620,-0.029999,-0.287712,0.954208,2.684158,-0.198532,2.535507,0.112469,3.165680
2024-11-25,0.940002,-8.049995,8.620010,-2.735687,0.03,-2.260000,21.139984,-0.538120,8.050140,-2.457336,...,1.159996,8.779266,1.830002,-0.127348,1.232521,-0.000008,1.181274,2.177521,-0.232695,3.824768


In [58]:
stock_growth_bool = stock_growth_between_month.dropna(axis=0).apply(lambda x: x >= 0)
stock_growth_bool

Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-04,True,True,True,True,False,True,True,True,False,True,...,True,False,True,False,True,True,True,True,False,True
2024-11-07,False,True,True,False,True,True,False,False,True,True,...,True,True,True,False,True,True,True,False,True,True
2024-11-10,False,False,True,False,False,True,False,True,True,False,...,True,False,False,True,False,False,False,False,False,True
2024-11-13,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2024-11-16,False,False,False,False,True,True,True,True,True,True,...,True,True,False,True,True,True,True,True,True,False
2024-11-19,True,False,False,True,False,True,True,True,False,True,...,True,True,True,True,True,True,False,True,True,True
2024-11-22,True,True,False,True,True,True,True,False,True,True,...,True,True,False,False,True,True,False,True,True,True
2024-11-25,True,False,True,False,True,False,True,False,True,False,...,True,True,True,False,True,False,True,True,False,True
2024-11-28,False,True,True,True,True,True,True,True,True,True,...,True,False,False,True,False,True,True,True,False,False


In [59]:
related_direction = data_for_pearson_matrix[target_ticker].loc[ticker_list]
related_direction = related_direction >= 0
related_direction

Ticker
META      True
CHX       True
POWL      True
VRSK      True
NFLX      True
AMZN      True
FAST      True
QCOM      True
IDXX      True
LIN       True
REGCO    False
CWST      True
EBAY      True
EQIX      True
MPWR      True
NHPAP     True
CSWI      True
MDGL      True
INFN      True
SOHOB     True
AMWD      True
UFPI      True
PAYX      True
JBHT      True
RVMD      True
LFUS      True
ROAD      True
ROST      True
VIASP     True
EXPE      True
EXC       True
MKSI      True
FRPT      True
TXN       True
KLAC      True
GWRS      True
BWEN      True
BYRN      True
COST      True
CME       True
TER       True
CTAS      True
BCPC      True
KOSS      True
ITRI      True
JJSF      True
FIP       True
CACC      True
ABNB      True
SSNC      True
ISRG      True
Name: ISRG, dtype: bool

In [60]:
target_stock_trend = stock_growth_bool[target_ticker]
related_stock_vote = stock_growth_bool.drop(columns=target_ticker)
related_stock_vote

Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-04,True,True,True,True,False,True,True,True,False,True,...,True,False,True,False,True,True,True,True,False,True
2024-11-07,False,True,True,False,True,True,False,False,True,True,...,True,True,True,False,True,True,True,False,True,True
2024-11-10,False,False,True,False,False,True,False,True,True,False,...,True,False,False,True,False,False,False,False,False,True
2024-11-13,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2024-11-16,False,False,False,False,True,True,True,True,True,True,...,True,True,False,True,True,True,True,True,True,False
2024-11-19,True,False,False,True,False,True,True,True,False,True,...,True,True,True,True,True,True,False,True,True,True
2024-11-22,True,True,False,True,True,True,True,False,True,True,...,True,True,False,False,True,True,False,True,True,True
2024-11-25,True,False,True,False,True,False,True,False,True,False,...,True,True,True,False,True,False,True,True,False,True
2024-11-28,False,True,True,True,True,True,True,True,True,True,...,True,False,False,True,False,True,True,True,False,False


In [61]:
for tick, bo in related_direction.items():
    if not bo:
        related_stock_vote[tick] = ~related_stock_vote[tick]
related_stock_vote

Ticker,ABNB,AMWD,AMZN,BCPC,BWEN,BYRN,CACC,CHX,CME,COST,...,ROAD,ROST,RVMD,SOHOB,SSNC,TER,TXN,UFPI,VIASP,VRSK
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-04,True,True,True,True,False,True,True,True,False,True,...,True,False,True,False,True,True,True,True,False,True
2024-11-07,False,True,True,False,True,True,False,False,True,True,...,True,True,True,False,True,True,True,False,True,True
2024-11-10,False,False,True,False,False,True,False,True,True,False,...,True,False,False,True,False,False,False,False,False,True
2024-11-13,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2024-11-16,False,False,False,False,True,True,True,True,True,True,...,True,True,False,True,True,True,True,True,True,False
2024-11-19,True,False,False,True,False,True,True,True,False,True,...,True,True,True,True,True,True,False,True,True,True
2024-11-22,True,True,False,True,True,True,True,False,True,True,...,True,True,False,False,True,True,False,True,True,True
2024-11-25,True,False,True,False,True,False,True,False,True,False,...,True,True,True,False,True,False,True,True,False,True
2024-11-28,False,True,True,True,True,True,True,True,True,True,...,True,False,False,True,False,True,True,True,False,False


In [62]:
related_stock_vote.mean(axis=1)

Date
2024-11-04    0.84
2024-11-07    0.62
2024-11-10    0.34
2024-11-13    0.12
2024-11-16    0.68
2024-11-19    0.76
2024-11-22    0.78
2024-11-25    0.62
2024-11-28    0.66
2024-12-01    0.40
2024-12-04    0.40
2024-12-07    0.40
2024-12-10    0.38
2024-12-13    0.30
2024-12-16    0.12
2024-12-19    0.62
2024-12-22    0.82
2024-12-25    0.30
2024-12-28    0.18
2024-12-31    0.44
2025-01-03    0.86
2025-01-06    0.56
2025-01-09    0.16
2025-01-12    0.68
2025-01-15    0.86
2025-01-21    0.54
2025-01-24    0.34
2025-01-27    0.44
2025-01-30    0.64
2025-02-02    0.56
2025-02-05    0.54
2025-02-08    0.74
2025-02-11    0.62
2025-02-14    0.44
2025-02-17    0.64
2025-02-20    0.16
2025-02-23    0.42
2025-02-26    0.50
dtype: float64

In [63]:
target_stock_trend

Date
2024-11-04     True
2024-11-07     True
2024-11-10     True
2024-11-13    False
2024-11-16     True
2024-11-19     True
2024-11-22    False
2024-11-25    False
2024-11-28     True
2024-12-01    False
2024-12-04     True
2024-12-07    False
2024-12-10     True
2024-12-13    False
2024-12-16    False
2024-12-19    False
2024-12-22     True
2024-12-25    False
2024-12-28    False
2024-12-31    False
2025-01-03     True
2025-01-06     True
2025-01-09     True
2025-01-12    False
2025-01-15     True
2025-01-21     True
2025-01-24    False
2025-01-27    False
2025-01-30    False
2025-02-02     True
2025-02-05     True
2025-02-08     True
2025-02-11    False
2025-02-14     True
2025-02-17     True
2025-02-20    False
2025-02-23    False
2025-02-26    False
Name: ISRG, dtype: bool

In [ ]:
def test_hypothesis(target, vote):
    results =  pd.concat([target, vote], axis=1, ignore_index=False)
    results.columns = ['target', 'vote']
    for thresh in [5]:
        decision = results['vote'] * 10 >= thresh
        accordance = results['target'] == decision
        roll = decision.shift(1)
        result = np.where(accordance, )

        results[f'decision_{thresh}'] = decision
        results[f'accordance_{thresh}'] = accordance
        results[f'roll_{thresh}'] = roll

    return results
    

In [73]:
test_hypothesis(target_stock_trend, related_stock_vote.mean(axis=1))

,target,vote,decision_5,accordance5,roll5
Date,,,,,
2024-11-04,True,0.84,True,True,NaN
2024-11-07,True,0.62,True,True,True
2024-11-10,True,0.34,False,False,True
2024-11-13,False,0.12,False,True,False
2024-11-16,True,0.68,True,True,False
2024-11-19,True,0.76,True,True,True
2024-11-22,False,0.78,True,False,True
2024-11-25,False,0.62,True,False,True
2024-11-28,True,0.66,True,True,True
